In [3]:
from __future__ import division
import pandas as pd
import os
import numpy as np
import itertools
import matplotlib.pyplot as plt
%matplotlib inline

from random import shuffle


# REVERSAL LEARNING TASK DESIGN FILE GENERATOR 

# Settings

design for behavioral project of follwing 7T fMRI project

- nr. blocks: 2 {2 runs of ca 16 min expected a TR of 2}
- nr of trials: 128 (per block) = 256 in total
- reversal point: after circa half of the trials but differently for each stimulus set

- sessions!!! Make sure to create design files for both sessions

### last edits

- reversal range increased to 26, 38 [per stim_set] - hence, (54, 76) instead of (60,68) from old design
- second session created
- stimuli adjusted for both sessions
- different reversal points for the different stimulus_sets
- number of trials adjusted to 128 [see also for reversal point]
- Phases_ renamed to phase names in final design generating loop (line 43)


#  TO DO

- change symbols for practice  ?



- make design for second session using galgolitsa alphabet √
- adjust symbols √
- ! will be created in preprocessing notebook ! Compute new 'Stim_high' 'Stim_low' columns that take into account the reversal

### Creates design files for the instrumental learning task

An experimental session containts two task, each of which is performed in three runs: 1. a vanilla instrumental learning task, and 2. a SAT-version of the task. Task order is counterbalanced, stimuli are updated between tasks. Stimuli are assigned a value randomly (i.e. also counterbalanced).


##### Design files are DataFrames with the following info per trial:
1. `stim_set`: Which stimulus set is presented? [0, 1, 2, 3, 4, 5, 6]
2. `correct_stim_lr`: What is the location (on the screen) of the winning stim? [0 = left, 1=right]
3. `p_win_left`: Probability of winning if left is chosen
4. `p_win_right'`: Probability of winning if right is chosen
5. `p_win_correct`: Probability of winning if correct answer is chosen
6. `p_win_incorrect`: Probability of winning if incorrect answer is chosen
7. `phase_x`: Duration for phase `x` (in s; see below)
8. `stim_left`, `stim_right`:  Stimuli that are presented left, right
9. `stim_high`, `stim_low`: Stimuli that correspond to the high, low probability of winning

##### Trial phases:
1. Fixation cross (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`
2. Cue (0.75s)
3. Fixation cross (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`
4. Stimulus (2s)
5. Stimulus choice (jittered timing) `[0.5, 0.75, 1.0, 1.25, 1.5s]`
6. Feedback (0.5s)
7. ITI (jittered timing) `[0.25, 0.5, ..., 3.25s]`

The timings mentioned above are for the MR version of the SAT task. Without MR, the jittered fixation crosses are skipped, the stimulus duration is not fixed to 2 seconds but ends when the participant gave a response, and highlight of choice is a fixed duration. ITI is shortened but still jittered a bit.

##### Potential future options:
1. Pseudorandomize trial order;
2. Optimize design (have code for this)

In [4]:
def get_settings(tr=2, verbose=True):
    p_win = [[.8, .2], [.7, .3]]            # removed , [.65, .35]
    n_runs = 2
    n_sessions = 1
    if tr == 2:
        n_trials = 128   #128
        jitters = [0.5, 0.75, 1, 1.25, 1.5]
        volumes_per_trial = 4
    
    n_trials_per_stimset = n_trials/len(p_win)
    trial_duration = volumes_per_trial*tr                # trial duration 
    total_duration = trial_duration*n_trials
    total_duration_min = total_duration/60
    total_volumes = 1 + n_trials*volumes_per_trial
    
    if verbose:
        print('Settings:\n\n\
        Sessions: {n_sessions}\n\
        Trials per run: {n_trials}\n\
        Assuming a TR of {tr} seconds\n\
        Jitter options: {jitters} seconds\n\
        Total duration: {tr}*{trial_duration}*{n_trials} = {total_duration} seconds = {total_duration_min} min\n\
        Total number of volumes necessary: 1+{n_trials}*{volumes_per_trial} = {total_volumes} + warm-up pulses'.format(**locals()))
        
    return({'jitter': jitters,
            'n_trials': n_trials,
            'ps': p_win})

In [5]:
def generate_block_design(n_trials, jitters, 
                          ps,
                          stim_sets=[0,1],
                          reversal_n_trials = (10,20),
                          mr_design=True, 
                          include_cue=False, 
                          phase_durations=['jittered', 0.75, 'jittered', 2, 'jittered', 0.75, 'iti'],
                          trial_duration=8,
                          practice=False):
    """ Generates design for a single block.
    
    jitters: list of possible jitter durations, in seconds (e.g., [0.5, 1, 1.5])
    mr_design: bool. If False, all phases that are jittered are set to -1 and skipped in the experiment; iti is set lower
    include_cue: bool. If False, the cue phase duration will be set to -1.
    """
    correct_stim_lr = [0, 1]
    if include_cue:
        cues = ['SPD', 'ACC']
    else:
        cues = ['']
    combs = list(itertools.product(stim_sets, correct_stim_lr, cues))
    
    # make basic df
    design = pd.DataFrame(combs * int(np.ceil((n_trials/len(combs)))), 
                          columns=['stimulus_set', 'correct_stim_lr', 'cue_txt'])
    # randomize
    design = design.sample(frac=1).reset_index(drop=True)#.reset_index('trial_ID')
    n_trials_real = design.shape[0] 
    design = design.iloc[:n_trials]
    
    if not n_trials == n_trials_real:
        print('WARNING: not totally balanced (%d not a multitude of %d)' %(n_trials, len(combs)))
    
    
    # SM
    for set_nr, stim_set in enumerate(stim_sets):
        print(stim_set, set_nr)
        if reversal_n_trials is not None:
            # how many trials is this stimulus shown?
            n_trials_this_stimset = design.loc[design.stimulus_set == stim_set].shape[0]
            reversal = np.zeros(n_trials_this_stimset)

            # determine reversal points: it's somwhere between reversal_n_trials[0] and reversal_n_trials[1]
            reversal_point = np.random.uniform(reversal_n_trials[0], reversal_n_trials[1], 1).astype(int)[0]

            # set all values after reversal point to 1
            reversal[reversal_point:] = 1
        else:
            reversal = 0
        # append to design
        design.loc[design['stimulus_set'] == stim_set, 'reversal'] = reversal
   

    # Add probabilities (left/right and correct/incorrect - this is redundant, I know)
    design['p_win_left'] = None
    design['p_win_right'] = None
    design['p_win_correct'] = None
    design['p_win_incorrect'] = None
    for stim_set, p_win in zip(stim_sets, ps):
        p_win_high, p_win_low = p_win[0], p_win[1]           
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 0), 'p_win_left'] = p_win_high
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 1), 'p_win_right'] = p_win_high
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 1), 'p_win_left'] = p_win_low
        design.loc[(design.stimulus_set==stim_set) & (design.correct_stim_lr == 0), 'p_win_right'] = p_win_low
        design.loc[(design.stimulus_set==stim_set), 'p_win_correct'] = p_win_high
        design.loc[(design.stimulus_set==stim_set), 'p_win_incorrect'] = p_win_low
    
    reversal_idx = design['reversal'] == 1
    # swap left & right probabilities
    design.loc[reversal_idx, 'p_win_left'], design.loc[reversal_idx, 'p_win_right'] = design.loc[reversal_idx, 'p_win_right'].copy(), design.loc[reversal_idx, 'p_win_left'].copy()
            
            
    # Add phase durations
    for phase, duration in enumerate(phase_durations):
        col_key = 'phase_' + str(phase+1)
        if duration == 'jittered':
            design[col_key] = np.random.choice(jitters, size=n_trials, replace=True)
        elif duration == 'iti':
            iti_phase_col_key = 'phase_' + str(phase+1)
        else:
            design[col_key] = duration
    
    design[iti_phase_col_key] = trial_duration - design[[col for col in design.columns if 'phase' in col]].apply(sum, axis=1)

    
    if not mr_design:
        # remove jitters
        for phase, duration in enumerate(phase_durations):
            col_key = 'phase_' + str(phase+1)
            if duration == 'jittered':
                design[col_key] = -.0001
            # set iti to randomly sampled from [0.5, 1]
            if duration == 'iti':
                design[col_key] = np.random.uniform(0.5, 1, design.shape[0])
        # set choice highlight phase to fixed 0.5
        design['phase_5'] = 0.5

    if not include_cue:
        # always skip cue
        design['phase_2'] = -.0001
        
    if practice and include_cue:
        design.cue.iloc[0] = 'ACC'

    return(design)

# generate_block_design(18, [0, 1, 2])

In [6]:
def get_task_type(run, subject_id, is_practice=False, is_debug=False):    
    def is_number(s):
        try:
            float(s)
            return True
        except ValueError:
            return False
    
    if is_debug:
        if run > 3:
            include_cue = False
        else:
            include_cue = False                        # set to False for RLWM study
    elif is_practice:
        # start practice without cues
        if run <= 2:  
            include_cue = False
        else:
            include_cue = False                        # set to False for RLWM study

    elif is_number(subject_id):
        if (int(subject_id) % 2) == 0:
            if run > 3:
                include_cue = False
            else:
                include_cue = False                        # set to False for RLWM study
        else:
            if run > 3:
                include_cue = False                        # set to False for RLWM study
            else:
                include_cue = False
                
    # Return stimuli
    if run <= 4:
        stim_sets = [0,1]
    elif run > 4:
        stim_sets = [3,4]
            
    return include_cue, stim_sets

In [7]:
def counterbalance_stimuli(subject_id, all_sets=  [[['h', 'i'],     #Cc
                                                   ['X', 'e']],    #fx
                                                  [['o', 'K'],     #jN
                                                   ['Y', 'y']]]):    
                           
                           
                           
                           
                           
#                                                    [[['h', 'i'],     #Cc
#                                                    ['X', 'e'],     #Dg
#                                                    ['u', 'n']],    #fx
#                                                   [['o', 'K'],     #jN
#                                                    ['Y', 'y'],     #QJ
#                                                    ['P', 'A']],    #mM
#                                                   [['V', 'v'],     #RS
#                                                    ['J', 'B'],     #sT
# 
        import itertools
        from copy import deepcopy

        n_shifts = [0, 1,3]  # add more/less if counterbalancing error when generating designs 
                            # [0,1] crashed for >9 subjects
                            # [0, 1,3] works for 100 subjects
        rev_inner = [False, True]
        switch_sets = [False, True]

        cb_df = pd.DataFrame(list(itertools.product(switch_sets, n_shifts, rev_inner)),
                             columns=['switch_sets', 'n_shifts', 'rev_inner'])
        cb_df['pp'] = np.arange(1, cb_df.shape[0] + 1)
        for set_n in range(1, 13):
            cb_df['stim_%d' % set_n] = None

        for pp in cb_df['pp']:
            idx = cb_df.pp == pp
            switch_sets = cb_df.loc[idx, 'switch_sets'].iloc[0]
            reverse_inner = cb_df.loc[idx, 'rev_inner'].iloc[0]
            n_shifts = cb_df.loc[idx, 'n_shifts'].iloc[0]

            if switch_sets:
                sets = deepcopy([all_sets[2:], all_sets[:2]])
            else:
                sets = deepcopy([all_sets[:2], all_sets[2:]])

            sets_allocated = 0
            for set_n, set_ in enumerate(sets):
                for i in range(n_shifts):
                    set_.insert(len(set_), set_.pop(0))  # move first item to last place

                if reverse_inner:
                    set_ = [x[::-1] for x in set_]  # reverse inner order

                # print('pp %d, %d, %s' % (pp, set_n, set_))
                #### NB: you could just use set_ as a final result; the placing in the dataframe and then reverting
                # back to a nested list is definitely not necessary but may help clarify what's going on here...
                for to_allocate in [0, 1]:
                    for to_allocate_i in [0, 1]:
                        cb_df.loc[idx, 'stim_%d' % (sets_allocated + 1)] = set_[to_allocate][to_allocate_i]
                        sets_allocated += 1

        pp_zero_based = int(subject_id) - 1
        row_iloc = int(pp_zero_based - np.floor(pp_zero_based / 12) * 12)
        colnames = cb_df.columns
        stim_list = cb_df.iloc[row_iloc][[x for x in colnames if 'stim' in x]].values.tolist()
        stim_nested_list = [[stim_list[0 + y * 2], stim_list[1 + y * 2]] for y in range(4)] # BEFORE: in range(6)
        print('Stimuli/set order for this pp: %s' % stim_nested_list)
        return stim_nested_list


In [6]:
# from random import shuffle
# stim_sets = [['A', 'B'], ['J', 'K'], ['M', 'N'], ['O', 'P'],  ['W', 'X'], ['c', 'e']]
# shuffle(stim_sets)
# print(stim_sets)

# for block in len(run):
#     if run == '1':
#         return stim_set = block_stim[0][1]
#     if run == '2':
#         return stim_set = block_stim[2][3]
#     if run == '3':
#         return stim_set = block_stim[4][5]
#     else:
#         return stim_sets = block_stim[6][7]
# /

# Create session 1 

agathodaimon alphabeth
 (1 = agathodaimon, 2 = galgolitsa)

In [11]:
def get_stimuli(subject_id, design):    
    if subject_id == 'PRACTICE':
        # The practice session always gets different stimuli
        stim_sets = [['j', 'm'], ['s', 'y'], ['u', 'n'], ['j', 'm'], ['s', 'y'], ['u', 'n']]
    else:
        stim_sets = [['h', 'i'],     #Cc
                    ['X', 'e'],     #Dg
#                     ['u', 'n'],    #fx
                    ['o', 'K'],     #jN
                    ['Y', 'y']]     #QJ

    if subject_id == 'DEBUG' or subject_id == 'PRACTICE':
        # just use counterbalancing order 1 for practice & debug
        subject_id = 1
    

       
    all_stim = counterbalance_stimuli(subject_id, all_sets=stim_sets)
    
    design['stim_left'] = None
    design['stim_right'] = None
    design['stim_high'] = None
    design['stim_low'] = None


    for i in range(len(all_stim)):
        idx_this_stim = design.stimulus_set == i
        design.loc[idx_this_stim, 'stim_high'] = all_stim[i][0]
        design.loc[idx_this_stim, 'stim_low'] = all_stim[i][1]

    for i in range(len(all_stim)):
        idx_this_stim = design.stimulus_set == i
        idx_this_stim_nonrev = (idx_this_stim) & (design.correct_stim_lr == 0)
        idx_this_stim_rev = (idx_this_stim) & (design.correct_stim_lr == 1)

        design.loc[idx_this_stim_nonrev, 'stim_left'] = design.loc[idx_this_stim_nonrev, 'stim_high']
        design.loc[idx_this_stim_nonrev, 'stim_right'] = design.loc[idx_this_stim_nonrev, 'stim_low']
        design.loc[idx_this_stim_rev, 'stim_left'] = design.loc[idx_this_stim_rev, 'stim_low']
        design.loc[idx_this_stim_rev, 'stim_right'] = design.loc[idx_this_stim_rev, 'stim_high']
   
    return design

In [12]:
tr = 2
n_subjects = 100
n_runs = 1      # set to one less than blocks you want 

save_dir = '../designs_reversal-learning'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.arange(1,n_subjects+1):
    designs_this_session = []
    print(subject_id)
    is_practice = subject_id == 'PRACTICE'
    is_debug = subject_id == 'DEBUG'
    
    for run in range(1,n_runs*2+1):
        settings = get_settings(tr=tr)
#        _, stim_set = get_task_type(run, subject_id, is_practice=is_practice, is_debug=is_debug)

        stim_set = [0, 1]
        include_cue = False
        
        # reversals
        reversals = (26, 38)
        
        print('Subject: %s, run: %d, cue: %s' %(subject_id, run, include_cue))
        design = generate_block_design(settings['n_trials'],
                                       settings['jitter'], 
                                       ps=settings['ps'],
                                       
                                       reversal_n_trials=reversals,
                                       
                                       stim_sets=stim_set,
                                       mr_design=False, 
                                       include_cue=include_cue)
        design['block'] = run
        design['stimulus_set'] = design['stimulus_set'] + 2*(run-1)
        designs_this_session.append(design)
        
    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design)
    
    
    design.rename(columns={'phase_1': 'fix_cross_1',
                           'phase_2': 'cue',
                           'phase_3': 'fix_cross_2',
                           'phase_4': 'stimulus',
                           'phase_5': 'highlight',
                           'phase_6': 'feedback',
                           'phase_7': 'iti_posttrial'}, 
                  inplace=True)
    
    fn = 'sub-' + str(subject_id).zfill(3) + '_ses-1_task-reversal-learning_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 1, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 1, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['h', 'i'], ['X', 'e'], ['o', 'K'], ['Y', 'y']]
sub-001_ses-1_task-reversal-learning_design
2
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 secon

Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 11, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['Y', 'y'], ['o', 'K'], ['X', 'e'], ['h', 'i']]
sub-011_ses-1_task-reversal-learning_design
12
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 12, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seco

Stimuli/set order for this pp: [['Y', 'y'], ['o', 'K'], ['X', 'e'], ['h', 'i']]
sub-021_ses-1_task-reversal-learning_design
22
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 22, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 22, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['y', 'Y'], ['K', 'o'], ['e', 'X'], ['i', 'h']]
sub-022_ses-1_task-reversal-learning_design
23
Settings:

        Sessions: 1
        Trials per run: 128
        A

Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 32, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['K', 'o'], ['y', 'Y'], ['i', 'h'], ['e', 'X']]
sub-032_ses-1_task-reversal-learning_design
33
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 33, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seco

(1, 1)
Stimuli/set order for this pp: [['e', 'X'], ['i', 'h'], ['y', 'Y'], ['K', 'o']]
sub-042_ses-1_task-reversal-learning_design
43
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 43, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 43, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['o', 'K'], ['Y', 'y'], ['h', 'i'], ['X', 'e']]
sub-043_ses-1_task-reversal-learning_design
44
Settings:

        Sessions: 1
        Trials per run: 128
  

(1, 1)
Stimuli/set order for this pp: [['X', 'e'], ['h', 'i'], ['Y', 'y'], ['o', 'K']]
sub-053_ses-1_task-reversal-learning_design
54
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 54, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 54, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['e', 'X'], ['i', 'h'], ['y', 'Y'], ['K', 'o']]
sub-054_ses-1_task-reversal-learning_design
55
Settings:

        Sessions: 1
        Trials per run: 128
  

Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 64, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['e', 'X'], ['i', 'h'], ['y', 'Y'], ['K', 'o']]
sub-064_ses-1_task-reversal-learning_design
65
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 65, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seco

(1, 1)
Stimuli/set order for this pp: [['i', 'h'], ['e', 'X'], ['K', 'o'], ['y', 'Y']]
sub-074_ses-1_task-reversal-learning_design
75
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 75, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 75, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['X', 'e'], ['h', 'i'], ['Y', 'y'], ['o', 'K']]
sub-075_ses-1_task-reversal-learning_design
76
Settings:

        Sessions: 1
        Trials per run: 128
  

Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 85, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['h', 'i'], ['X', 'e'], ['o', 'K'], ['Y', 'y']]
sub-085_ses-1_task-reversal-learning_design
86
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 86, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seco

(1, 1)
Stimuli/set order for this pp: [['Y', 'y'], ['o', 'K'], ['X', 'e'], ['h', 'i']]
sub-095_ses-1_task-reversal-learning_design
96
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 96, run: 1, cue: False
(0, 0)
(1, 1)
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.0666666667 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 96, run: 2, cue: False
(0, 0)
(1, 1)
Stimuli/set order for this pp: [['y', 'Y'], ['K', 'o'], ['e', 'X'], ['i', 'h']]
sub-096_ses-1_task-reversal-learning_design
97
Settings:

        Sessions: 1
        Trials per run: 128
  

In [13]:
## np.mean(design['iti_posttrial'])

In [14]:
#print all lines 

#show all columns
with pd.option_context('display.max_rows', None):
    display(design)

,stimulus_set,correct_stim_lr,cue_txt,reversal,p_win_left,p_win_right,p_win_correct,p_win_incorrect,fix_cross_1,cue,fix_cross_2,stimulus,highlight,feedback,iti_posttrial,block,stim_left,stim_right,stim_high,stim_low
0,0,0,,0.0,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.934636,1,e,X,e,X
1,0,1,,0.0,0.2,0.8,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.943830,1,X,e,e,X
2,1,1,,0.0,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.660877,1,h,i,i,h
3,1,0,,0.0,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.838227,1,i,h,i,h
4,1,0,,0.0,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.586646,1,i,h,i,h
5,0,0,,0.0,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.654744,1,e,X,e,X
6,0,1,,0.0,0.2,0.8,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.793930,1,X,e,e,X
7,0,1,,0.0,0.2,0.8,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.858111,1,X,e,e,X
8,0,0,,0.0,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.588453,1,e,X,e,X
9,0,1,,0.0,0.2,0.8,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.551439,1,X,e,e,X


# Create session 2 

galgolitsa alphabeth
 (1 = agathodaimon, 2 = galgolitsa)

In [8]:
def get_stimuli(subject_id, design):    
    if subject_id == 'PRACTICE':
        # The practice session always gets different stimuli
        stim_sets = [['j', 'm'], ['s', 'y'], ['u', 'n'], ['j', 'm'], ['s', 'y'], ['u', 'n']]
    else:
        stim_sets = [['É','Ï'],
                    ['K', 'h'],
                    ['y', 'z'],
                    ['L', 'v']]
            
    if subject_id == 'DEBUG' or subject_id == 'PRACTICE':
        # just use counterbalancing order 1 for practice & debug
        subject_id = 1
       
    all_stim = counterbalance_stimuli(subject_id, all_sets=stim_sets)
    design['stim_left'] = None
    design['stim_right'] = None
    design['stim_high'] = None
    design['stim_low'] = None


    for i in range(len(all_stim)):
        idx_this_stim = design.stimulus_set == i
        design.loc[idx_this_stim, 'stim_high'] = all_stim[i][0]
        design.loc[idx_this_stim, 'stim_low'] = all_stim[i][1]

    for i in range(len(all_stim)):
        idx_this_stim = design.stimulus_set == i
        idx_this_stim_nonrev = (idx_this_stim) & (design.correct_stim_lr == 0)
        idx_this_stim_rev = (idx_this_stim) & (design.correct_stim_lr == 1)

        design.loc[idx_this_stim_nonrev, 'stim_left'] = design.loc[idx_this_stim_nonrev, 'stim_high']
        design.loc[idx_this_stim_nonrev, 'stim_right'] = design.loc[idx_this_stim_nonrev, 'stim_low']
        design.loc[idx_this_stim_rev, 'stim_left'] = design.loc[idx_this_stim_rev, 'stim_low']
        design.loc[idx_this_stim_rev, 'stim_right'] = design.loc[idx_this_stim_rev, 'stim_high']
   
    return design

In [9]:
tr = 2
n_subjects = 100
n_runs = 1      # set to one less than blocks you want 

save_dir = '../designs_reversal-learning'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.arange(1,n_subjects+1):
    designs_this_session = []
    print(subject_id)
    is_practice = subject_id == 'PRACTICE'
    is_debug = subject_id == 'DEBUG'
    
    for run in range(1,n_runs*2+1):
        settings = get_settings(tr=tr)
#        _, stim_set = get_task_type(run, subject_id, is_practice=is_practice, is_debug=is_debug)

        stim_set = [0, 1]
        include_cue = False
        
        # reversals
        reversals = (26, 38)
        
        print('Subject: %s, run: %d, cue: %s' %(subject_id, run, include_cue))
        design = generate_block_design(settings['n_trials'],
                                       settings['jitter'], 
                                       ps=settings['ps'],
                                       
                                       reversal_n_trials=reversals,
                                       
                                       stim_sets=stim_set,
                                       mr_design=False, 
                                       include_cue=include_cue)
        design['block'] = run
        design['stimulus_set'] = design['stimulus_set'] + 2*(run-1)
        designs_this_session.append(design)
        
    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design)
    
    
    design.rename(columns={'phase_1': 'fix_cross_1',
                           'phase_2': 'cue',
                           'phase_3': 'fix_cross_2',
                           'phase_4': 'stimulus',
                           'phase_5': 'highlight',
                           'phase_6': 'feedback',
                           'phase_7': 'iti_posttrial'}, 
                  inplace=True)
    
    fn = 'sub-' + str(subject_id).zfill(3) + '_ses-2_task-reversal-learning_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 1, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 1, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['É', 'Ï'], ['K', 'h'], ['y', 'z'], ['L', 'v']]
sub-001_ses-2_task-reversal-learning_design
2
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds

Stimuli/set order for this pp: [['L', 'v'], ['y', 'z'], ['K', 'h'], ['É', 'Ï']]
sub-011_ses-2_task-reversal-learning_design
12
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 12, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 12, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['v', 'L'], ['z', 'y'], ['h', 'K'], ['Ï', 'É']]
sub-012_ses-2_task-reversal-learning_design
13
Settings:

        Sessions: 1
        Trials per run: 128
        Ass

Stimuli/set order for this pp: [['v', 'L'], ['z', 'y'], ['h', 'K'], ['Ï', 'É']]
sub-022_ses-2_task-reversal-learning_design
23
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 23, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 23, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['L', 'v'], ['y', 'z'], ['K', 'h'], ['É', 'Ï']]
sub-023_ses-2_task-reversal-learning_design
24
Settings:

        Sessions: 1
        Trials per run: 128
        Ass

Stimuli/set order for this pp: [['L', 'v'], ['y', 'z'], ['K', 'h'], ['É', 'Ï']]
sub-033_ses-2_task-reversal-learning_design
34
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 34, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 34, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['v', 'L'], ['z', 'y'], ['h', 'K'], ['Ï', 'É']]
sub-034_ses-2_task-reversal-learning_design
35
Settings:

        Sessions: 1
        Trials per run: 128
        Ass

1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 44, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['z', 'y'], ['v', 'L'], ['Ï', 'É'], ['h', 'K']]
sub-044_ses-2_task-reversal-learning_design
45
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 45, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 se

Stimuli/set order for this pp: [['h', 'K'], ['Ï', 'É'], ['v', 'L'], ['z', 'y']]
sub-054_ses-2_task-reversal-learning_design
55
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 55, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 55, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['y', 'z'], ['L', 'v'], ['É', 'Ï'], ['K', 'h']]
sub-055_ses-2_task-reversal-learning_design
56
Settings:

        Sessions: 1
        Trials per run: 128
        Ass

Stimuli/set order for this pp: [['K', 'h'], ['É', 'Ï'], ['L', 'v'], ['y', 'z']]
sub-065_ses-2_task-reversal-learning_design
66
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 66, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 66, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['h', 'K'], ['Ï', 'É'], ['v', 'L'], ['z', 'y']]
sub-066_ses-2_task-reversal-learning_design
67
Settings:

        Sessions: 1
        Trials per run: 128
        Ass

Stimuli/set order for this pp: [['h', 'K'], ['Ï', 'É'], ['v', 'L'], ['z', 'y']]
sub-076_ses-2_task-reversal-learning_design
77
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 77, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 77, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['K', 'h'], ['É', 'Ï'], ['L', 'v'], ['y', 'z']]
sub-077_ses-2_task-reversal-learning_design
78
Settings:

        Sessions: 1
        Trials per run: 128
        Ass

Stimuli/set order for this pp: [['K', 'h'], ['É', 'Ï'], ['L', 'v'], ['y', 'z']]
sub-087_ses-2_task-reversal-learning_design
88
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 88, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 88, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['h', 'K'], ['Ï', 'É'], ['v', 'L'], ['z', 'y']]
sub-088_ses-2_task-reversal-learning_design
89
Settings:

        Sessions: 1
        Trials per run: 128
        Ass

Stimuli/set order for this pp: [['Ï', 'É'], ['h', 'K'], ['z', 'y'], ['v', 'L']]
sub-098_ses-2_task-reversal-learning_design
99
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 99, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: 99, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['K', 'h'], ['É', 'Ï'], ['L', 'v'], ['y', 'z']]
sub-099_ses-2_task-reversal-learning_design
100
Settings:

        Sessions: 1
        Trials per run: 128
        As

In [17]:
design.head()

,stimulus_set,correct_stim_lr,cue_txt,reversal,p_win_left,p_win_right,p_win_correct,p_win_incorrect,fix_cross_1,cue,fix_cross_2,stimulus,highlight,feedback,iti_posttrial,block,stim_left,stim_right,stim_high,stim_low
0,0,0,,0.0,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.617054,1,h,K,h,K
1,1,1,,0.0,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.578859,1,É,Ï,Ï,É
2,1,1,,0.0,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.620712,1,É,Ï,Ï,É
3,1,0,,0.0,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.835271,1,Ï,É,Ï,É
4,0,0,,0.0,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.756896,1,h,K,h,K


In [18]:
design.tail()

,stimulus_set,correct_stim_lr,cue_txt,reversal,p_win_left,p_win_right,p_win_correct,p_win_incorrect,fix_cross_1,cue,fix_cross_2,stimulus,highlight,feedback,iti_posttrial,block,stim_left,stim_right,stim_high,stim_low
123,3,0,,1.0,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.802601,2,z,y,z,y
124,3,0,,1.0,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.925072,2,z,y,z,y
125,3,1,,1.0,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.662206,2,y,z,z,y
126,3,0,,1.0,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.842994,2,z,y,z,y
127,3,1,,1.0,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.846191,2,y,z,z,y


________________________________________________________________________________________________________________

________________________________________________________________________________________________________________

________________________________________________________________________________________________________________

________________________________ below not needed _______________________________________________________________

________________________________________________________________________________________________________________

________________________________________________________________________________________________________________

## For debug, we create a nice a short session

In [44]:
tr = 2
n_subjects = 1
n_runs = 1

save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.hstack(['DEBUG']):
    designs_this_session = []
    is_practice = subject_id == 'PRACTICE'
    is_debug = subject_id == 'DEBUG'
    
    for run in range(1,n_runs*2+1):
        settings = get_settings(tr=tr)
        include_cue, stim_set = get_task_type(run, subject_id, is_practice=is_practice, is_debug=is_debug)
        print('Subject: %s, run: %d, cue: %s' %(subject_id, run, include_cue))
        design = generate_block_design(6, settings['jitter'], ps=settings['ps'],
                                       stim_sets=stim_set,
                                       mr_design=False, include_cue=include_cue)
        design['block'] = run
        designs_this_session.append(design)
    
    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design)
    fn = 'sub-' + str(subject_id).zfill(2) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: DEBUG, run: 1, cue: False
0 0
1 1
Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: DEBUG, run: 2, cue: False
0 0
1 1
Stimuli/set order for this pp: [['h', 'i'], ['X', 'e'], ['u', 'n'], ['o', 'K'], ['Y', 'y'], ['P', 'A']]
sub-DEBUG_design


In [81]:
design.head()

,stimulus_set,correct_stim_lr,cue_txt,reversal,p_win_left,p_win_right,p_win_correct,p_win_incorrect,fix_cross_1,cue,fix_cross_2,stimulus,highlight,feedback,iti_posttrial,block,stim_left,stim_right,stim_high,stim_low
0,1,1,,0.0,0.3,0.7,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.518997,1,d,L,L,d
1,1,0,,0.0,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.977827,1,L,d,L,d
2,0,0,,0.0,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.534103,1,y,z,y,z
3,1,0,,0.0,0.7,0.3,0.7,0.3,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.786755,1,L,d,L,d
4,0,0,,0.0,0.8,0.2,0.8,0.2,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.903140,1,y,z,y,z


In [120]:
#print all lines 

#show all columns
with pd.option_context('display.max_rows', None):
    display(design)


,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,reversal,phase_1,phase_2,phase_3,phase_4,phase_5,phase_6,phase_7,block,stim_left,stim_right,stim_high,stim_low
0,1,0,,0.7,0.3,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.726047,1,J,K,J,K
1,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.661562,1,K,J,J,K
2,0,0,,0.8,0.2,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.898201,1,A,B,A,B
3,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.954166,1,K,J,J,K
4,0,0,,0.8,0.2,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.722564,1,A,B,A,B
5,1,0,,0.7,0.3,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.803111,1,J,K,J,K
6,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.952858,1,B,A,A,B
7,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.671067,1,K,J,J,K
8,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.869663,1,B,A,A,B
9,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.512313,1,B,A,A,B


### Idem for the practice session

In [43]:
tr = 2
n_runs = 1


save_dir = '../designs'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
    
for subject_id in np.hstack(['PRACTICE']):
    designs_this_session = []
    is_practice = subject_id == 'PRACTICE'
    is_debug = subject_id == 'DEBUG'
    
    for run in [1]:
        settings = get_settings(tr=tr)
        include_cue, stim_set = get_task_type(run, subject_id, is_practice=is_practice, is_debug=is_debug)
        print('Subject: %s, run: %d, cue: %s' %(subject_id, run, include_cue))
        design = generate_block_design(12, settings['jitter'], ps=settings['ps'],
                                       stim_sets=stim_set,
                                       mr_design=False, include_cue=include_cue, practice=True)
        design['block'] = run
        designs_this_session.append(design)
    
    design = pd.concat(designs_this_session)
    design = get_stimuli(subject_id=subject_id, design=design)
    fn = 'sub-' + str(subject_id).zfill(2) + '_design'
    print(fn)
    design.to_csv(save_dir + '/' + fn + '.csv', sep='\t', index_label='trial_ID')

Settings:

        Sessions: 1
        Trials per run: 128
        Assuming a TR of 2 seconds
        Jitter options: [0.5, 0.75, 1, 1.25, 1.5] seconds
        Total duration: 2*8*128 = 1024 seconds = 17.066666666666666 min
        Total number of volumes necessary: 1+128*4 = 513 + warm-up pulses
Subject: PRACTICE, run: 1, cue: False
0 0
1 1
Stimuli/set order for this pp: [['j', 'm'], ['s', 'y'], ['u', 'n'], ['j', 'm'], ['s', 'y'], ['u', 'n']]
sub-PRACTICE_design


In [118]:
design

,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,reversal,phase_1,phase_2,phase_3,phase_4,phase_5,phase_6,phase_7,block,stim_left,stim_right,stim_high,stim_low
0,1,0,,0.7,0.3,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.726047,1,J,K,J,K
1,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.661562,1,K,J,J,K
2,0,0,,0.8,0.2,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.898201,1,A,B,A,B
3,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.954166,1,K,J,J,K
4,0,0,,0.8,0.2,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.722564,1,A,B,A,B
5,1,0,,0.7,0.3,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.803111,1,J,K,J,K
6,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.952858,1,B,A,A,B
7,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.671067,1,K,J,J,K
8,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.869663,1,B,A,A,B
9,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.512313,1,B,A,A,B


In [86]:
idx = design.block == 2

In [87]:
idx 

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10     True
11     True
Name: block, dtype: bool

In [68]:
1 < np.cumsum(idx) < 2

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [20]:
idx = design.block == 1
#idx & np.cumsum(idx) < 4

In [21]:
idx = design.block==1
design.loc[idx].groupby(['stimulus_set'])[['stim_left', 'stim_right']].last().reset_index()[['stim_left', 'stim_right']].values.tolist()

[['B', 'A'], ['K', 'J']]

In [143]:
design[['stim_left', 'stim_right']].values

array([['j', 'm'],
       ['m', 'j'],
       ['s', 'y'],
       ['y', 's'],
       ['m', 'j'],
       ['m', 'j'],
       ['s', 'y'],
       ['j', 'm'],
       ['y', 's'],
       ['s', 'y'],
       ['j', 'm'],
       ['y', 's'],
       ['y', 's'],
       ['m', 'j'],
       ['m', 'j'],
       ['j', 'm'],
       ['j', 'm'],
       ['y', 's'],
       ['s', 'y'],
       ['y', 's'],
       ['j', 'm'],
       ['s', 'y'],
       ['m', 'j'],
       ['s', 'y'],
       ['j', 'm'],
       ['s', 'y'],
       ['j', 'm'],
       ['s', 'y'],
       ['y', 's'],
       ['m', 'j'],
       ['y', 's'],
       ['y', 's'],
       ['m', 'j'],
       ['m', 'j'],
       ['s', 'y'],
       ['j', 'm'],
       ['m', 'j'],
       ['j', 'm'],
       ['s', 'y'],
       ['y', 's'],
       ['s', 'y'],
       ['m', 'j'],
       ['j', 'm'],
       ['j', 'm'],
       ['y', 's'],
       ['y', 's'],
       ['s', 'y'],
       ['m', 'j']], dtype=object)

In [20]:
design['stim_right'].uni

AttributeError: 'Series' object has no attribute 'uni'

In [119]:
#show all columns
with pd.option_context('display.max_rows', None):
    display(design)


,stimulus_set,correct_stim_lr,cue,p_win_left,p_win_right,p_win_correct,p_win_incorrect,reversal,phase_1,phase_2,phase_3,phase_4,phase_5,phase_6,phase_7,block,stim_left,stim_right,stim_high,stim_low
0,1,0,,0.7,0.3,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.726047,1,J,K,J,K
1,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.661562,1,K,J,J,K
2,0,0,,0.8,0.2,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.898201,1,A,B,A,B
3,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.954166,1,K,J,J,K
4,0,0,,0.8,0.2,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.722564,1,A,B,A,B
5,1,0,,0.7,0.3,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.803111,1,J,K,J,K
6,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.952858,1,B,A,A,B
7,1,1,,0.3,0.7,0.7,0.3,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.671067,1,K,J,J,K
8,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.869663,1,B,A,A,B
9,0,1,,0.2,0.8,0.8,0.2,0.0,-0.0001,-0.0001,-0.0001,2,0.5,0.75,0.512313,1,B,A,A,B
